In [1]:
!pip install pyyaml==5.1

In [2]:
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html

In [4]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [5]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
import os
import pickle


In [6]:
# https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md

In [7]:
config_file_path = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
checkpoint_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

output_dir = "./" 
num_classes = 1

cfg_save_path = "OD_cfg.pickle"

In [8]:
device = "cuda" # "cpu"

In [9]:
train_dataset_name = "CD_train"
train_images_path = "../input/indian-currency/ten/ten"
train_json_annotations = "../input/indian-currency/train.json"

test_dataset_name = "CD_test"
test_images_path = "../input/indian-currency/test"
test_json_annotations = "../input/indian-currency/test.json"

In [10]:
register_coco_instances(name = train_dataset_name, metadata = {}, 
                       json_file = train_json_annotations, image_root = train_images_path)

register_coco_instances(name = test_dataset_name, metadata = {}, 
                       json_file = test_json_annotations, image_root = test_images_path)

In [11]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2 import model_zoo
import random
import cv2
import matplotlib.pyplot as plt

def plot_sample(dataset_name, n=1):
    dataset_custom = DatasetCatalog.get(dataset_name)
    dataset_custom_metadata = MetadataCatalog.get(dataset_name)
    
    for s in random.sample(dataset_custom, n):
        img = cv2.imread(s["file_name"])
        v = visualizer(img[:,:,::-1], metadata = dataset_custom_metadata, scale = 0.5)
        v = v.draw_dataset_dict(s)
        plt.figure(figsize=(15,20))
        plt.imshow(v.get_image())
        plt.show()

In [12]:
def get_train_cfg(config_file_path, checkpoint_url, train_dataset_name, test_dataset_name, num_classes, device, output_dir):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(config_file_path))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
    cfg.DATASETS.TRAIN = (train_dataset_name, )
    cfg.DATASETS.TEST = (test_dataset_name, )
    
    cfg.DATALOADER.NUM_WORKERS = 2
    
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = 1000
    cfg.SOLVER.STEPS = []
    
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    cfg.MODEL.DEVICE = device
    cfg.OUTPUT_DIR = output_dir
    
    return cfg

In [13]:
# MAIN

cfg = get_train_cfg(config_file_path, checkpoint_url, train_dataset_name, test_dataset_name, num_classes, device, output_dir)

with open(cfg_save_path, 'wb') as f:
    pickle.dump(cfg, f, protocol = pickle.HIGHEST_PROTOCOL)
    
#     os.mkdirs(cfg.OUTPUT_DIR, exist_ok=True)
    
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    
    trainer.train()
    
    

Testing

In [21]:
def on_image(image_path,predictor):
    im = cv2.imread(image_path)
    outputs = predictor(im)
    v = Visualizer(im[:,:,::-1], metadata={}, scale=0.5, instance_mode=ColorMode.SEGMENTATION)
    v=v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    
    plt.figure(figsize=(14,10))
    plt.imshow(v.get_image())
    plt.show()

    
def on_video(videoPath, predictor):
    cap = cv2.VideoCapture(videoPath)
    if(cap.isOpened()==False):
        print("Error opening file ... ")
        return
    
    (success,image) = cap.read()
    while success:
        predictions = predictor(image)
        v = Visualizer(image[:,:,::-1], metadata={}, instance_mode=ColorMode.SEGMENTATION)
        output = v.draw_instance_predictions(predictions["instances"].to("cpu"))
        
        
        cv2.imshow("Result", output.get_image()[:,:,::-1])
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        (success,image) = cap.read()

In [27]:
from detectron2.engine import DefaultPredictor

import os
import pickle


cfg_save_path = "OD_cfg.pickle"

with open(cfg_save_path,'rb') as f:
    cfg = pickle.load(f)

    
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH = 0.5
    
    
predictor = DefaultPredictor(cfg)


image_path = "../input/indian-currency/test/tenold127.jpg"

#not required but if we have a video feed we can test this
#videoPath = "test/"

on_image(image_path,predictor)

#on_video(videoPath, predictor)